In [96]:
import sys
sys.path.append('/home/ivan/distribution_connector')
import os

from sklearn.decomposition import TruncatedSVD

In [2]:
from sklearn import datasets
import numpy as np
import torch
from tqdm import tqdm
from connector_utils import test_models, gather_statistics, test_func
import matplotlib.pyplot as plt

In [3]:
from connector import Connector
# from one_layer_utils import samples, make_dataset, get_model, get_b
from utils import test_model

In [4]:
import models
architecture = getattr(models, "Linear3NoBias") #LinearOneLayer LogRegression

import data
loaders, num_classes = data.loaders(
    "CIFAR10",
    "data",
    1024,
    1,
    "VGG",
    True,
    shuffle_train=False)

model1 = architecture.base(num_classes=10, **architecture.kwargs)
model2 = architecture.base(num_classes=10, **architecture.kwargs)

model1.load_state_dict(torch.load('curves/Linear3NoBias/curve1/checkpoint-100.pt')['model_state'])
model2.load_state_dict(torch.load('curves/Linear3NoBias/curve2/checkpoint-100.pt')['model_state'])


Files already downloaded and verified
You are going to run models on the test set. Are you sure?
Files already downloaded and verified


IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [5]:
model1.cuda();
model1.eval();
test_model(model1, loaders, cuda=True)

train results {'nll': 0.45399812195777894, 'loss': 0.45399812195777894, 'accuracy': 84.832}
test results {'nll': 1.2863960428237915, 'loss': 1.2863960428237915, 'accuracy': 60.34}


({'nll': 0.45399812195777894, 'loss': 0.45399812195777894, 'accuracy': 84.832},
 {'nll': 1.2863960428237915, 'loss': 1.2863960428237915, 'accuracy': 60.34})

In [6]:
def get_model(W, architecture, bias=False, per_layer=True):
    model_sampled = architecture.base(num_classes=10, **architecture.kwargs)
    model_samples = np.array(W)  

    if per_layer:
        for parameter, w in zip(model_sampled.parameters(), W):
            parameter.data.copy_(torch.from_numpy(w))
    else:
        for i, parameter in enumerate(model_sampled.parameters()):
            w = W[i//2]
            if i % 2 == 0:
                offset = 0
            N = parameter.data.shape[1]
            w_part = w[:, offset:offset+N]
            offset = N
            if i % 2 == 0:
                parameter.data.copy_(torch.from_numpy(w_part))
            else:
                parameter.data.copy_(torch.from_numpy(w_part.T))
            

    return model_sampled

def samples(model):
    p = [list(model.parameters())[i].data.cpu().numpy() for i in range(len(list(model.parameters())))]
    return p

def samples_per_layer(model, bias=True):
    p = samples(model)
    if bias:
        p = [np.hstack([p[i], p[i+1][:, None]]) for i in range(0, len(p), 2)]        
    return p

In [7]:
layer = 0
model1.cuda()
model1.eval()
functions = []
targ = []
for X, y in loaders['test']:
    functions.append(model1(X.cuda(), N=layer))
    targ.append(y)

print('l', functions[0].shape)
targ = np.concatenate(targ)
funcs = np.concatenate([p.cpu().data.numpy() for p in functions])

l torch.Size([1024, 6144])


In [8]:
def get_X(data='test'):
    dataset = []
    for X, y in loaders[data]:
        dataset.append(X)

    dataset = np.concatenate([p.view(-1,3*32*32).cpu().data.numpy() for p in dataset])
    
    return dataset

In [9]:
data = get_X()

In [10]:
data.shape

(10000, 3072)

In [11]:
def get_funcs(model1, data='test', layer=0):
    model1.cuda()
    model1.eval()
    functions = []
    for X, y in loaders[data]:
        functions.append(model1(X.cuda(), N=layer))

    funcs = np.concatenate([p.cpu().data.numpy() for p in functions])
    
    return funcs

def get_alpha(funcs, V):
    alpha = funcs.T @ V.T
    return alpha

def get_betas(funcs, V):
    betas = np.linalg.inv(funcs.T @ V.T)
#     f_inv = np.linalg.pinv(funcs.T)
    betas = V @ f_inv
    return betas

def accuracy(pred):
    ens_acc = 100.0 * np.mean(np.argmax(pred, axis=1) == targ)
    print(ens_acc)

In [102]:
f_inv = np.linalg.pinv(funcs.T)

In [103]:
f_inv2 = np.linalg.pinv(funcs2.T)

In [104]:
f_inv.shape

(10000, 6144)

In [105]:
funcs.T.shape, f_inv.shape

((6144, 10000), (10000, 6144))

In [106]:
# (f_inv @ funcs.T).shape

In [107]:
funcs = get_funcs(model1, data='test')
funcs2  = get_funcs(model2, data='test')

# funcs2 = funcs2[:10000]

In [108]:
funcs.shape, funcs2.shape

((10000, 6144), (10000, 6144))

In [89]:
U_init, Sigma_init, V_init = np.linalg.svd(funcs.T)

In [16]:
Sigma_init.shape

(6144,)

In [243]:
U_init0, Sigma_init0, V_init0 =  np.linalg.svd(data.T)

In [64]:
U_init2, Sigma_init2, V_init2 = np.linalg.svd(funcs2.T)

In [66]:
V_init2.T @ V_init

array([[ 0.09596302,  0.00412305, -0.03508306, ..., -0.00094693,
        -0.01262829,  0.00524393],
       [-0.08779276,  0.03962382,  0.07021306, ...,  0.01515507,
        -0.01558601, -0.01281927],
       [ 0.05324541, -0.0160114 , -0.05011205, ..., -0.0056352 ,
         0.01498281,  0.0052655 ],
       ...,
       [-0.01893221, -0.01234936,  0.00509776, ...,  0.16162594,
        -0.01053133, -0.01030669],
       [ 0.00291595,  0.00819505, -0.01673825, ..., -0.00081445,
         0.08962141, -0.00716509],
       [ 0.00310175,  0.01450393,  0.00831493, ...,  0.01208472,
         0.01484879,  0.14524095]], dtype=float32)

In [74]:
# s1 = np.zeros(10000)
# s1[:6144] = Sigma_init
# s2 = np.zeros(10000)
# s2[:6144] = 1

In [73]:
# (V_init @ V_init2.T) @ (s2 * V_init2)

In [90]:
U_init.shape, Sigma_init.shape, V_init.shape

((6144, 6144), (6144,), (10000, 10000))

In [91]:
# (U_init * Sigma_init) @ V_init


In [92]:
comp = 6144 #6144
U, Sigma, V = U_init[:, :comp], Sigma_init[:comp], V_init[:comp]

In [93]:
V.shape

(6144, 10000)

In [195]:
V = np.eye(10000)[:6144]

In [220]:
alpha1 = get_alpha(funcs, V)
alpha2 = get_alpha(funcs2, V)

In [221]:
alpha1.shape

(6144, 6144)

In [222]:
betas1 = get_betas(funcs, V)
betas2 =  get_betas(funcs2, V)

In [223]:
# betas1@f

In [35]:
# betas1.shape

In [36]:
W10 = list(model1.parameters())[0].data.cpu().numpy()
W20 = list(model2.parameters())[0].data.cpu().numpy()

In [37]:
W10.shape, W20.shape

((6144, 3072), (6144, 3072))

In [38]:
U_init0.shape

NameError: name 'U_init0' is not defined

In [250]:
W10_n = W10 @ U_init0
W20_n = W20 @ U_init0

In [257]:
np.linalg.svd(W10)[1]

array([6.544975  , 5.4749255 , 5.3777485 , ..., 0.19082779, 0.18902187,
       0.18870509], dtype=float32)

In [258]:
np.linalg.svd(W20)[1]

array([6.4968715 , 5.515618  , 5.3739834 , ..., 0.19163038, 0.1903389 ,
       0.18875374], dtype=float32)

In [255]:
np.linalg.svd(W10_n)[1]

array([6.544975  , 5.4749255 , 5.3777485 , ..., 0.19082779, 0.18902187,
       0.18870507], dtype=float32)

In [256]:
np.linalg.svd(W20_n)[1]

array([6.4968715 , 5.515618  , 5.3739834 , ..., 0.19163038, 0.1903389 ,
       0.18875374], dtype=float32)

In [109]:
pred = W12 @ np.maximum(W11 @ np.maximum(W10 @ data.T, 0), 0)
accuracy(pred.T)

60.34


In [114]:
pred = W12 @ np.maximum(W11 @ funcs.T, 0)
accuracy(pred.T)

60.34


In [116]:
W21 @ funcs2.T @ f_inv

array([[ 0.00209087, -0.01183992, -0.03358892, ..., -0.01675107,
         0.00135146, -0.01833726],
       [-0.01591677, -0.00552126,  0.04036836, ..., -0.04225153,
         0.01808026, -0.00903646],
       [-0.00392423,  0.02599351, -0.00139501, ...,  0.0003346 ,
        -0.01566672,  0.03327934],
       ...,
       [-0.01516421, -0.00625636,  0.00496992, ..., -0.01260684,
         0.0340665 , -0.04136172],
       [-0.00196367,  0.01392346, -0.02947181, ...,  0.00024454,
        -0.06954266,  0.03346093],
       [-0.01038512, -0.00194788,  0.0028613 , ...,  0.00840465,
         0.04643633, -0.02032827]], dtype=float32)

In [128]:
pred = Connector(W12.T, W22.T).arc_connect(t=0.5)[1].T @ np.maximum(Connector((W21 @ funcs2.T @ f_inv), W11).arc_connect(t=0.5)[1] @  funcs.T, 0)
accuracy(pred.T)

52.12


In [78]:
pred = W12 @ np.maximum(W11 @ funcs.T @ V.T @ V, 0)
accuracy(pred.T)

60.34


In [85]:
pred = ( W22) @ np.maximum((W21 @ funcs.T @ V.T) @ V, 0)
accuracy(pred.T)

8.780000000000001


In [77]:
funcs.T.shape, V.shape

((6144, 10000), (6144, 10000))

In [45]:
pred = W12 @ np.maximum(W11 @ funcs.T, 0)
accuracy(pred.T)

60.34


In [59]:
def accuracy(pred, N=10000):
    ens_acc = 100.0 * np.mean(np.argmax(pred[:N], axis=1) == targ[:N])
    print(ens_acc)
    
pred = W12 @ np.maximum(W11 @ funcs.T[:, :6144] @  f_inv @ funcs.T, 0)
accuracy(pred.T)

60.33


In [65]:
def accuracy(pred, N=10000):
    ens_acc = 100.0 * np.mean(np.argmax(pred[:N], axis=1) == targ[:N])
    print(ens_acc)
    
pred = W12 @ np.maximum(W11 @ funcs.T[:, :6144] @  f_inv @ funcs.T, 0)
accuracy(pred.T)

60.33


In [98]:
pred = W12 @ np.maximum(W11 @ funcs.T[:, :] @  f_inv @ funcs.T, 0)
accuracy(pred.T)

60.34


In [ ]:
pred = W12 @ np.maximum(W11 @ funcs.T[:, :] @  f_inv @ funcs.T, 0)
accuracy(pred.T)

In [99]:
pred = W22 @ np.maximum(W21 @ funcs2.T[:, :] @  f_inv @ funcs.T, 0)
accuracy(pred.T)

60.050000000000004


In [101]:
Wn1 = W11 @ funcs.T[:, :] @  f_inv 
Wn2 = W21 @ funcs2.T[:, :] @  f_inv 
cntr = Connector(Wn1, Wn2)
Wn = cntr.arc_connect(t=0.5)[1]

pred = W22 @ np.maximum(Wn @ funcs.T, 0)
accuracy(pred.T)

45.72


In [97]:
f_inv.shape

(10000, 6144)

In [72]:
pred.shape

(10, 10000)

In [61]:
    
pred = W22 @ np.maximum(W21 @ funcs2.T, 0)
accuracy(pred.T)

60.419999999999995


In [68]:
    
pred = W22 @ np.maximum(W21 @ funcs2.T[:, :6144] @ f_inv @ funcs.T[:, :6144] , 0)
accuracy(pred.T, N=6144) 

60.758463541666664


In [70]:
    
pred = W22 @ np.maximum(W21 @ funcs2.T[:, :6144] @ f_inv @ funcs.T[:, :] , 0)
accuracy(pred.T) 

42.07


In [64]:
funcs.T.shape, W11.shape, f_inv.shape, funcs2.T.shape

((6144, 10000), (2000, 6144), (6144, 6144), (6144, 10000))

In [49]:
# f_inv = np.linalg.inv(funcs.T[:, :6144])

In [278]:
W10_n* Sigma_init0

array([[-6.9393394e+01, -3.6944726e+00, -2.5505659e-01, ...,
        -1.5883979e-03,  3.4995936e-03, -1.1651417e-03],
       [-1.8697100e+01, -2.1300530e-01, -1.1048281e+01, ...,
        -4.2271642e-03,  2.0189788e-03, -4.7966675e-04],
       [-6.5082275e+01, -8.4967957e+00, -7.8768616e+01, ...,
        -1.2863235e-04,  1.5900825e-03, -2.6306489e-03],
       ...,
       [-9.8739281e+01, -2.7278234e+01, -6.1139725e+01, ...,
         1.7832149e-03, -1.2008103e-03,  4.6887463e-03],
       [-3.4055332e+01,  1.4878210e+01, -2.6090891e+01, ...,
         7.7544688e-03,  5.6731161e-03, -4.3090750e-03],
       [ 1.9557550e+01,  1.3253250e+01, -2.7822254e+01, ...,
        -1.9443581e-03,  1.5113211e-03,  2.5002116e-03]], dtype=float32)

In [282]:
pred = W12 @ np.maximum(W11 @ np.maximum(((W10_n+W20_n)/2 * Sigma_init0) @ V_init0[:len(Sigma_init0)], 0), 0)
accuracy(pred.T)

48.93


In [ ]:
Connector(W10_n, W20_n).arc_connect(t=0.5)[1]

In [288]:
cntr = Connector(W10_n, W20_n)
Wh0 = cntr.arc_connect(t=0.5)[1]
pred = W12 @ np.maximum(W11 @ np.maximum((Wh0 * Sigma_init0) @ V_init0[:len(Sigma_init0)], 0), 0)
accuracy(pred.T)

48.699999999999996


In [286]:
cntr = Connector(W10.T, W20.T)
Wh0 = cntr.arc_connect(t=0.5)[1].T
pred = W12 @ np.maximum(W11 @ np.maximum(Wh0 @ data.T, 0), 0)
accuracy(pred.T)

49.07


In [39]:
W11 = list(model1.parameters())[-2].data.cpu().numpy()
W21 = list(model2.parameters())[-2].data.cpu().numpy()

In [40]:
W12 = list(model1.parameters())[-1].data.cpu().numpy()
W22 = list(model2.parameters())[-1].data.cpu().numpy()

In [41]:
Wmodel1 = samples_per_layer(model1, bias=False)
Wmodel2 = samples_per_layer(model2, bias=False)

In [229]:
Wmodel = []
for i, (S1, S2) in enumerate(zip(Wmodel1, Wmodel2)):
    if i==0:
        print(S1.T.shape)
        cntr = Connector(S1.T, S2.T)
        S = cntr.lin_connect(t=0.5)[1].T
    else:
        print(S1.shape)
        cntr = Connector(S1, S2)
        S = cntr.lin_connect(t=0.5)[1]
    Wmodel.append(S)

(3072, 6144)
(2000, 6144)
(10, 2000)


In [230]:
model = get_model(Wmodel, architecture, bias=False, per_layer=True)
funcs3 =  get_funcs(model, data='test')

In [231]:
model.cuda();
model.eval();
test_model(model, loaders, cuda=True)

train results {'nll': 1.9967549339294433, 'loss': 1.9967549339294433, 'accuracy': 51.482}
test results {'nll': 2.162593546295166, 'loss': 2.162593546295166, 'accuracy': 29.76}


({'nll': 1.9967549339294433, 'loss': 1.9967549339294433, 'accuracy': 51.482},
 {'nll': 2.162593546295166, 'loss': 2.162593546295166, 'accuracy': 29.76})

In [232]:
alpha3 = get_alpha(funcs3, V) 
betas3 =  get_betas(funcs3, V)
# W31 = list(model.parameters())[-2].data.cpu().numpy()
# W31 = list(model.parameters())[-2].data.cpu().numpy()

In [233]:
# pred = np.maximum(funcs3 @ betas3.T @ alpha2.T @ W21.T, 0) @ W22.T
# accuracy(pred)

In [267]:
pred = np.maximum(funcs3 @ betas3.T @ alpha2.T @ W21.T, 0) @ W22.T
accuracy(pred)

39.86


In [235]:
pred = np.maximum(funcs3 @ betas3.T @ alpha1.T @ W11.T, 0) @ W12.T
accuracy(pred)

48.93


In [236]:
Wh1 = alpha1.T @ W11.T
Wh2 = alpha2.T @ W21.T
cntr = Connector(Wh1.T, Wh2.T)
Wh = cntr.arc_connect(t=0.5)[1].T

In [237]:
pred = np.maximum(funcs @ betas1.T @ Wh, 0) @ (W12.T+W22.T)/2
accuracy(pred)

51.76


In [238]:
pred = np.maximum(funcs2 @ betas2.T @ (alpha1.T @ W11.T + alpha2.T @ W21.T)/2, 0) @ (W12.T+W22.T)/2
accuracy(pred)

9.51


In [239]:
pred = np.maximum(funcs2 @ betas2.T @ (alpha1.T @ W11.T + alpha1.T @ W11.T)/2, 0) @ (W12.T+W22.T)/2
accuracy(pred)

11.91


In [268]:
pred = np.maximum(funcs @ W11.T, 0) @ W12.T
accuracy(pred)
pred = np.maximum(funcs2 @ W21.T, 0) @ W22.T
accuracy(pred)
pred = np.maximum(funcs @ betas1.T @ alpha1.T @ W11.T, 0) @ W12.T
accuracy(pred)
pred = np.maximum(funcs2 @ betas2.T @ alpha2.T @ W21.T, 0) @ W22.T
accuracy(pred)
pred = np.maximum(funcs @ betas1.T @ alpha2.T @ W21.T, 0) @ W22.T
accuracy(pred)
pred = np.maximum(funcs2 @ betas2.T  @ alpha1.T @ W11.T, 0) @ W12.T
accuracy(pred)

60.34
60.419999999999995
60.34
9.32
60.050000000000004
11.26


In [241]:
g1 = get_funcs(model1, data='test', layer=1)
g2 = get_funcs(model2, data='test', layer=1)


In [130]:
Wh1 = alpha1.T @ W11.T
Wh2 = alpha2.T @ W21.T
cntr = Connector(Wh1.T, Wh2.T)
Wh = cntr.arc_connect(t=0.5)[1].T
print(Wh1.shape)

g3 = np.maximum(funcs2 @ betas2.T @ Wh, 0)

NameError: name 'alpha1' is not defined

In [157]:
U_init2, Sigma_init2, V_init2 = np.linalg.svd(g1.T)

In [158]:
U_init2.shape, Sigma_init2.shape, V_init2.shape

((2000, 2000), (2000,), (10000, 10000))

In [159]:
comp = 10000
U2, Sigma2, V2 = U_init2[:, :comp], Sigma_init2[:comp], V_init2[:comp]

In [160]:
V2 = np.eye(10000)

In [161]:
alpha12 = get_alpha(g1, V2)
alpha22 = get_alpha(g2, V2)

betas12 = get_betas(g1, V2)
betas22 =  get_betas(g2, V2)


In [162]:
alpha32 = get_alpha(g3, V2) 
betas32 =  get_betas(g3, V2)

In [163]:
pred = g3 @ betas32.T @ alpha12.T @ W12.T
accuracy(pred)

58.45


In [164]:
pred = g3 @ betas32.T @ alpha22.T @ W22.T
accuracy(pred)

58.45


In [129]:
pred = g3 @ betas32.T @ (alpha12.T @ W12.T + alpha22.T @ W22.T)/2
accuracy(pred)

NameError: name 'g3' is not defined

In [180]:
Wh1 = alpha12.T @ W12.T
Wh2 = alpha22.T @ W22.T
cntr = Connector(Wh1.T, Wh2.T)
Whh = cntr.arc_connect(t=0.5)[1].T
pred = g3 @ betas32.T @ Whh
accuracy(pred)

59.309999999999995


In [178]:
pred = g3 @ betas32.T @ Whh
accuracy(pred)

59.150000000000006


In [130]:
pred = g1 @ betas12.T @ (alpha12.T @ W12.T + alpha22.T @ W22.T)/2
accuracy(pred)

60.86


In [131]:
pred = g1 @ betas12.T @ alpha12.T @ W12.T
accuracy(pred)

60.34


In [132]:
pred = g2 @ betas22.T @ alpha12.T @ W12.T
accuracy(pred)

59.95


In [133]:
pred = g2 @ betas22.T @ alpha22.T @ W22.T
accuracy(pred)

60.419999999999995


In [134]:
pred = g1 @ betas12.T @ alpha22.T @ W22.T
accuracy(pred)

59.699999999999996
